<a href="https://colab.research.google.com/github/intelligent-environments-lab/occupant_centric_grid_interactive_buildings_course/blob/main/src/notebooks/tutorials/sqlite_relational_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relational database: SQLite
---

This tutorial introduces the concepts of a relational database which is the a popular approach to storing tabular data that have meaningful relationships. 

You are most likely more familiar with using flat files e.g. `.csv` files to store and retrieve data. `.csv` files are excellent for storing and retrieving modestly large data and can be easily inspected in a text editor without needing to install or know how to use special software. They are also easily transferable from one user or machine to another without the risk of losing access or corruption. 

However, flat files start to fail when the data size becomes large and you need to quickly query it to extract information. They also fail when one needs to enforce strict data integrity measures e.g., only allowing specific data type in a column, only allowing specific categories in a column, making sure that numeric values are positive or making sure that no row is duplicated to mention a few of these measures. Also, making edits to a large-sized flat file can be difficult as it requires that you read the entire file into memory, make the edit then write back to disk. Furthermore, if you have a group of flat files that are related in someway, you will need to invent and maintain a method to keep track of those relationships in a way that is understood by you the designer but also anyone else who make use of the dataset. An example of this will be one `.csv` file that holds metadata for a group of buildings and another that holds electricity consumption time series for the buildings. You may choose to define a `building_id` column in both files that helps link the two however, a change in the column name in either file or changing the id in any row of either file will break this relationship without any repercussions. These issues with flat files escalate when you work in large teams and have many people accessing the same flat file at the same time.

There are workarounds to solve some of the flat file issues while still providing some of its features. For example, Excel lets you define types for columns however, there is a limit to the number of rows a single Excel sheet can hold and this design feature can have very [serious consequences](https://www.youtube.com/watch?v=-NuTlczV72Q&t=349s). Excel also let's you use cell references and formulas to define dynamic values but I am sure we all know how ugly those formulas get with complicated calculation engineers carry out! You can also choose to store your data as a `.parquet`, `.pickle` or `.hdf` file all of which solve some of the data integrity issues however, these file types either easily get corrupted moving from one machine to another or require that you still read the entire file into memory to make minor edits or pull specific rows. There are spin-offs of the `pandas` library e.g. `dask` which eliminate the need to load entire datasets into memory to execute queries however they solve only a few of these problems at best.

At the core, any representation of data can be though of as being a database. Thus choosing to store data in a `.txt`, `.csv`, `.xlsx`, or `.json`. e.t.c. format all have the same outcome: creation of a database. However, depending on how we plan to interface with the data, we might choose to one format over the other. A [relational database](https://en.wikipedia.org/wiki/Relational_database) is a database based on the [relational model](https://en.wikipedia.org/wiki/Relational_model) where data is stored in tuples and a group of tuples (think rows) form a relation (think table). Tables can then be linked by way of keys. With a relational database, you do not need to read entire tables into memory to make queries as the database management system handles the overhead behind the scenes. You also get to define data types when you declare the tables as well as any constraints that must be applied to the values entered into a column so that you ensure integrity in your data. by use of primary and foreign keys, you can maintain persistent links between tables. An important characteristic of relational database is that they possess [ACID](https://en.wikipedia.org/wiki/ACID) (atomicity, consistency, isolation, durability) properties that ensure transactions on the database either succeed completely or fail completely even in edge cases like power outage or system crashes (atomicity), transactions adhere to the rules of the database (consistency), transactions are independent of simultaneously executed transactions (isolation) and successful transactions are persistent in memory even in a case of system failure (durability). All these reinforce the integrity of relational databases. Once the database structure is defined by way of a [schema](https://en.wikipedia.org/wiki/Database_schema), you can carry out [CRUD](https://en.wikipedia.org/wiki/Create,_read,_update_and_delete) (create, read, update, delete) operations in a transaction. Almost all relational databases are written in [Structured Query Language](https://en.wikipedia.org/wiki/SQL) (SQL) that for the most part, reads like human language.

There is a number of engines that run on the relational model. Think of these engines as different 'operating systems' that allow you set up relations. Some of the popular ones are Oracle Database, MySQL, Microsoft SQL Server, PostgreSQL, Microsoft Access, MariaDB and SQLite.

The summary above is by no means an exhaustive introduction to relational databases as it is an entire course of its own in reality however, this tutorial will provide you with just enough information to start write your own SQL queries using the [SQLite](https://www.sqlite.org/about.html) database engine. We choose SQLite here to introduce the relational database because unlike the other engines, SQLite does not require server maintenance as it resides in your local disk like any other flat file and can be copied, pasted and shared like any other file you are used to working with. It is the most widely deployed database engine found in almost all consumer electronics and yet it is free! In terms of limitations, a SQLite database can hold u to 281 TB of data and supports most frequently used statements in the more robust engines. It also comes preinstalled with Python so all we need to do is import the `sqlite3` library.

The remainder of this tutorial will teach operations and concepts of relational databases using the SQLite engine.

## Setting up environment
---

To begin, let us install third-party libraries we will be needing. We will make use of `pandas` to help us return our queries in tabular form:

In [1]:
# ********* UNCOMMENT CODE BELOW TO INSTALL DEPENDENCIES *********
# !pip install pandas

Now we can import the Python libraries we will use. `sqlite3` is the in-built Python SQLite library while `os` allows us perform system I/O operations:

In [2]:
import os
import sqlite3
from typing import List, Mapping, Tuple, Union
import pandas as pd

## Selecting and loading the dataset
---

We will use data from the [The Building Data Genome Project 2, energy meter data from the ASHRAE Great Energy Predictor III competition](https://www.nature.com/articles/s41597-020-00712-x) to drive our point about relational databases. Specifically, the building metadata file. The urls for these files are defined below:

In [3]:
METADATA_URL = 'https://media.githubusercontent.com/media/buds-lab/building-data-genome-project-2/master/data/metadata/metadata.csv'

First we read in the metadata and view its structure:

In [4]:
building_metadata = pd.read_csv(METADATA_URL)

with pd.option_context('display.max_columns', None):
    display(building_metadata.head())
    display(building_metadata.info())

,building_id,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,lat,lng,timezone,electricity,hotwater,chilledwater,steam,water,irrigation,solar,gas,industry,subindustry,heatingtype,yearbuilt,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating
0,Panther_lodging_Dean,Panther,NaN,0.0,Lodging/residential,Residence Hall,508.8,5477.0,28.517689,-81.379039,US/Eastern,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,1989.0,NaN,NaN,NaN,NaN,271,NaN,NaN,None,NaN
1,Panther_lodging_Shelia,Panther,NaN,0.0,Lodging/residential,Residence Hall,929.0,10000.0,28.517689,-81.379039,US/Eastern,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,1992.0,NaN,NaN,NaN,NaN,62,NaN,NaN,None,NaN
2,Panther_lodging_Ricky,Panther,NaN,0.0,Lodging/residential,Residence Hall,483.1,5200.0,28.517689,-81.379039,US/Eastern,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,2016.0,NaN,NaN,NaN,NaN,534,NaN,NaN,None,NaN
3,Panther_education_Rosalie,Panther,0.0,0.0,Education,Research,690.5,7432.0,28.517689,-81.379039,US/Eastern,Yes,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,2008.0,NaN,NaN,NaN,NaN,276,NaN,NaN,None,NaN
4,Panther_education_Misty,Panther,1.0,0.0,Education,Research,252.7,2720.0,28.517689,-81.379039,US/Eastern,Yes,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,NaN,NaN,NaN,NaN,375,NaN,NaN,None,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   building_id            1636 non-null   object 
 1   site_id                1636 non-null   object 
 2   building_id_kaggle     1449 non-null   float64
 3   site_id_kaggle         1599 non-null   float64
 4   primaryspaceusage      1615 non-null   object 
 5   sub_primaryspaceusage  1615 non-null   object 
 6   sqm                    1636 non-null   float64
 7   sqft                   1636 non-null   float64
 8   lat                    1399 non-null   float64
 9   lng                    1399 non-null   float64
 10  timezone               1636 non-null   object 
 11  electricity            1578 non-null   object 
 12  hotwater               185 non-null    object 
 13  chilledwater           555 non-null    object 
 14  steam                  370 non-null    object 
 15  wate

None

While there are 31 columns in this dataset, we will only make use of the `building_id`, `site_id`, `primaryspaceusage`, `sqft`, `solar`, and `yearbuilt` columns:

In [5]:
building_metadata = building_metadata[['building_id', 'site_id', 'primaryspaceusage', 'sqft', 'solar', 'yearbuilt']].copy()
display(building_metadata.head())

,building_id,site_id,primaryspaceusage,sqft,solar,yearbuilt
0,Panther_lodging_Dean,Panther,Lodging/residential,5477.0,NaN,1989.0
1,Panther_lodging_Shelia,Panther,Lodging/residential,10000.0,NaN,1992.0
2,Panther_lodging_Ricky,Panther,Lodging/residential,5200.0,NaN,2016.0
3,Panther_education_Rosalie,Panther,Education,7432.0,NaN,2008.0
4,Panther_education_Misty,Panther,Education,2720.0,NaN,2004.0


We should also extract some quick summary statistics about this dataset using the `describe` method:

In [6]:
display(building_metadata.describe(include='all'))

,building_id,site_id,primaryspaceusage,sqft,solar,yearbuilt
count,1636,1636,1615,1636.000000,5,817.000000
unique,1636,19,16,NaN,1,NaN
top,Panther_lodging_Dean,Rat,Education,NaN,Yes,NaN
freq,1,305,617,NaN,5,NaN
mean,NaN,NaN,NaN,88522.795232,NaN,1968.236230
std,NaN,NaN,NaN,108178.666361,NaN,30.935801
min,NaN,NaN,NaN,283.000000,NaN,1900.000000
25%,NaN,NaN,NaN,20849.000000,NaN,1949.000000
50%,NaN,NaN,NaN,54620.500000,NaN,1971.000000
75%,NaN,NaN,NaN,111521.000000,NaN,1994.000000


So we have a dataset of 1,636 buildings all with unique `building_id` form 19 unique sites comprising 16 unique primary space uses. The buildings are built 1900 to 2017 with an average total floor area of 88,522 ft<sup>2</sup>. Only 5 buildings have solar PVs.

# Initializing the database
---

To _Pythonically_ initialize a SQLite database all we do is call the `sqlite3.connect` method while supplying a filepath. The extension used on the filename is open ended but we will use `sqlite3` here:

In [7]:
DATABASE_FILEPATH = os.path.join('building-data-genome-project-2.sqlite3')

# first delete any existing database
if os.path.exists(DATABASE_FILEPATH):
    os.remove(DATABASE_FILEPATH)

else:
    pass

# create new database
database = sqlite3.connect(DATABASE_FILEPATH)

However, we really only want to connect when we have a transaction to carry out if not we might risk locking the database and preventing other transactions from happening. To end a connection, we use the `close` method:

In [8]:
database.close()

## Defining a table
---

Now we have the database initialized, we can start by defining a `TABLE` to hold our metadata. To define a `TABLE` we use the `CREATE TABLE` keywords and execute on our database. Note that SQL is not case-sensitive however, for readability, SQL keywords are always upper-cased. 

In [9]:
transaction = """
CREATE TABLE IF NOT EXISTS building_metadata (
    building_id TEXT,
    site_id TEXT,
    primary_space_usage TEXT,
    floor_area REAL,
    year_built INTEGER,
    has_solar TEXT,
    PRIMARY KEY (building_id)
);
"""

The above query when executed, will create a `TABLE` called `building_metadata` that has six columns. In SQLite, there are 5 [data types](https://www.sqlite.org/datatype3.html) a column can have namely:

1. `NULL`: The value is a NULL value.
2. `INTEGER`: The value is a signed integer, stored in 0, 1, 2, 3, 4, 6, or 8 bytes depending on the magnitude of the value.
3. `REAL`: The value is a floating point value, stored as an 8-byte IEEE floating point number.
4. `TEXT`: The value is a text string, stored using the database encoding (UTF-8, UTF-16BE or UTF-16LE).
5. `BLOB`: The value is a blob of data, stored exactly as it was input.

The `TABLE` we have used makes use of three of these types to store our metadata which types similar to the original pandas dataframe. In relational databases, the `PRIMARY KEY` indicates a column whose values are all unique thus, is suitable for use as the unique identifier for each record in the `TABLE`. Later, we we learn about `FOREIGN KEY` and how it can be used to define relationships to another `TABLE` that has a matching `PRIMARY KEY`. We can now execute our `CREATE TABLE` query. Since we want to make sure we always close or database after execution, it makes sense to define function for executing queries that handles this. We will also

In [10]:
def execute_transaction(database_filepath: str, transaction: str) -> Tuple[List[List[str]], List[List[tuple]]]:
    column_names = []
    records = []

    # get each statement (query) for the transaction
    statements = transaction.split(';')

    # include statement that ensure foreign key relationships are honored
    statements = ['PRAGMA foreign_keys = ON'] + statements

    try:
        # connect to database
        database = sqlite3.connect(database_filepath)

        # cursor used to execute statements
        cursor = database.cursor()

        for s in statements:
            cursor.execute(s)

            # collect any data returned from the executed statment
            if cursor.description is not None:
                column_names.append([d[0] for d in cursor.description])
                records.append(cursor.fetchall())
            else:
                pass
        
        # commit the transaction to the database otherwise it does not
        # get saved when the database closes
        database.commit()
    
    finally:
        # close all connections
        cursor.close()
        database.close()

    return column_names, records

Finally, create the `TABLE`:

In [11]:
_ = execute_transaction(DATABASE_FILEPATH, transaction)

## Insert record into table
---

Now that we have created our `TABLE`, we can proceed to insert records in our `TABLE`. We will start simple by inserting our own made up records to highlight some considerations when defining a `TABLE`. `INSERT INTO <table_name> (<column_name>, ...) VALUES (<column_value>, ...)` syntax is used to put records in a `TABLE`. See the example below:

In [12]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built, has_solar)
VALUES
    ('ECJ', 'UT Austin Main Campus', 'Academic', 240246, 1974, NULL)
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

We have inserted one record into our `TABLE` and in the next section we will retrieve this data.

## Read records from table
---

To read records from a `TABLE`, we use the `SELECT <column_name>, ... FROM <table_name>`. Alternatively, you can replace the `<column_name>, ...` with `*` to return all columns in the `TABLE`. However, it is better to be explicit when writing queries so we will always use the former approach:

In [13]:
transaction = """
SELECT
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
column_names, records = execute_transaction(DATABASE_FILEPATH, transaction)
print('Column names:', column_names)
print('Records:', records)

Column names: [['building_id', 'site_id', 'primary_space_usage', 'floor_area', 'year_built', 'has_solar']]
Records: [[('ECJ', 'UT Austin Main Campus', 'Academic', 240246.0, 1974, None)]]


We can then load it into a `pandas.DataFrame` object: 

In [14]:
data = pd.DataFrame(records[0], columns=column_names[0])
display(data)

,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,ECJ,UT Austin Main Campus,Academic,240246.0,1974,None


It also makes sense then to define a function that returns a `pandas.DataFrame` when we want to read a `TABLE`. We will give it a generic name because later on, we will use it to read a special representation of a `TABLE` called `VIEW`:

In [15]:
def read_database(database_filepath: str, transaction: str) -> pd.DataFrame:
    column_names, records = execute_transaction(database_filepath, transaction)

    return pd.DataFrame(records[-1], columns=column_names[-1])

We can now use our generic `read_database` function to retrieve specific columns from our `bldg_metadata` `TABLE`:

In [16]:
transaction = """
SELECT
    building_id, 
    site_id, 
    primary_space_usage,
    has_solar
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,building_id,site_id,primary_space_usage,has_solar
0,ECJ,UT Austin Main Campus,Academic,None


## Updating table record
---

We can update the value of any column or columns using the `UPDATE <table_name> SET <column_name> = <value>, ...`:

In [17]:
transaction = """
UPDATE building_metadata
SET
    building_id = 'ETC'       
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

transaction = """
SELECT
    building_id, 
    site_id
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,building_id,site_id
0,ETC,UT Austin Main Campus


Alternatively, you can use the `WHERE` keyword to update only records that match specific criteria:

In [18]:
transaction = """
UPDATE building_metadata
SET
    building_id = 'RLM'
WHERE
    site_id = 'UT Austin Pickle Research Campus'       
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

transaction = """
SELECT
    building_id, 
    site_id
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,building_id,site_id
0,ETC,UT Austin Main Campus


Above, we have intentionally used a `site_id` that does not match any record in the database in our `WHERE` clause so that we do not alter the `building_id`.

Note that queries can be written all in one line as SQL is not indentation or line break sensitive. For readability and debugging purposes, you may choose to break the query as you please. In the next section we will learn how to delete a record from a `TABLE`.

## Delete table record
---

We use the `DELETE FROM <table_name> WHERE <column_name> = <value>` to delete specific records. Alternatively, you can exclude the `WHERE` clause to delete all records from the `TABLE`:

In [19]:
transaction = """
DELETE FROM building_metadata       
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

transaction = """
SELECT
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar


We have now emptied out our table but it still remains defined in our schema.

So far so good! Our table is loosely defined to some degree. Asides the data types we enforced, it isn't much different from a `.csv` database. For example, we inserted a record were the `has_solar` value was `NULL`. Sometimes we want to make sure that every value in a records ins `NOT NULL` for data integrity sake. To fix this, we will need to redefine our table. With other advanced database engines, you can add constraints even after tables have been created however, SQLite does not have that robustness. 

However, even the advanced engines have limitations on how much you can alter the schema after creation which brings to light an important lesson about relational databases. One must take out some time to think about the best way to define the database schema such that it always ensures data integrity and is scalable in the case that the data grows in size and content. You do not want a situation where you have to redefine a table that already has many existing relations!

In the next section, we will drop the `building_metadata` table and redefine it to prevent `NULL` values.

## Drop a table from schema
---

To drop a table, we just use the `DROP TABLE IF EXISTS <table_name>` syntax:

In [20]:
transaction = """
DROP TABLE IF EXISTS building_metadata;      
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

In the next section, we will redefine the `building_metadata` `TABLE` to always require that a non-`NULL` value is supplied for each column in a record.

## Adding constraints to table definition
---

### The `NOT NULL` constraint

The `NOT NULL` constraint accompanies the column definition when defining a `TABLE`. See example below:

In [21]:
transaction = """
CREATE TABLE IF NOT EXISTS building_metadata (
    building_id TEXT NOT NULL,
    site_id TEXT NOT NULL,
    primary_space_usage TEXT NOT NULL,
    floor_area REAL NOT NULL,
    year_built INTEGER NOT NULL,
    has_solar TEXT NOT NULL,
    PRIMARY KEY (building_id)
);
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

Now, if we try to insert our former records, we will get an error:

In [22]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built, has_solar)
VALUES
    ('ECJ', 'UT Austin Main Campus', 'Academic', 240246, 1974, NULL)
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

IntegrityError: NOT NULL constraint failed: building_metadata.has_solar

Alternatively, we can define the table in such a way that if no value is provided for any column, a default is assigned using the `DEFAULT` keyword:

In [23]:
transaction = """
DROP TABLE IF EXISTS building_metadata;
CREATE TABLE IF NOT EXISTS building_metadata (
    building_id TEXT NOT NULL,
    site_id TEXT NOT NULL,
    primary_space_usage TEXT NOT NULL,
    floor_area REAL NOT NULL,
    year_built INTEGER NOT NULL,
    has_solar TEXT NOT NULL DEFAULT 'No',
    PRIMARY KEY (building_id)
);
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Main Campus', 'Academic', 240246, 1974);
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction) 

Notice that we excluded `has_solar` in the `INSERT` statement. See what is returned when we read the table:

In [24]:
transaction = """
SELECT
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,ECJ,UT Austin Main Campus,Academic,240246.0,1974,No


The record was assigned the default value for `has_solar` since no value was provided during the `INSERT` statement. The `NOT NULL` keyword is very powerful as you can see to ensure your data is never incomplete.

### The `UNIQUE` constraint

Another constraint we might want to consider when defining our tables is to keep certain column values or group of column values unique to avoid duplication in our data. Using the record we have worked with so far, we do not expect to have the same `building_id` in the same `site_id` however, the way we have defined our table allows such violation when inserting new records. We can fix this by including the `UNIQUE` constraint when defining our `TABLE`:

In [25]:
transaction = """
DROP TABLE IF EXISTS building_metadata;
CREATE TABLE IF NOT EXISTS building_metadata (
    building_id TEXT NOT NULL,
    site_id TEXT NOT NULL,
    primary_space_usage TEXT NOT NULL,
    floor_area REAL NOT NULL,
    year_built INTEGER NOT NULL,
    has_solar TEXT NOT NULL DEFAULT 'No',
    PRIMARY KEY (building_id),
    UNIQUE (building_id, site_id)
);
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Main Campus', 'Academic', 240246, 1974);
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

transaction = """
SELECT
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,ECJ,UT Austin Main Campus,Academic,240246.0,1974,No


Now, any attempt to insert the existing `building_id`-`site_id` pair will fail even if all other column values are different:

In [26]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Main Campus', 'Laboratory', 740246, 1980)
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

IntegrityError: UNIQUE constraint failed: building_metadata.building_id, building_metadata.site_id

Can we insert a record that has `building_id` that matches an existing record?

In [27]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Pickle Research Campus', 'Laboratory', 740246, 1980)
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

IntegrityError: UNIQUE constraint failed: building_metadata.building_id

Turns out that we can't! Any idea why?

The reason is that we use `building_id` as the `PRIMARY KEY` when defining our `TABLE`. `PRIMARY KEY` by definition must be unique as it is used as the unique identifier for any record. This highlights a design flaw in our current `TABLE` definition. It is possible that the same building name exists in two different sites. For example, every state has a Capitol Building. However, with the way we have designed our `TABLE`, we can only insert one record ever with `building_id` = `Capitol Building`! Yet, we still want to make sure that for the same `site_id`, there are no two buildings with the same `building_id`. 

The better way to define our `TABLE` is to use a truly unique `id` as the `PRIMARY KEY`. It is typical and safe approach is to use an `INTEGER` as unique ID for every table you create. There is a way to create a composite `PRIMARY KEY` where multiple two or more columns make up the key which has the advantage of automatically creating indexes for you that makes queries faster. However, sometimes it may be difficult to decide in the beginning what group of columns are always unique.

We will now redefine our table to use an `INTEGER` as the `PRIMARY KEY`:

In [28]:
transaction = """
DROP TABLE IF EXISTS building_metadata;
CREATE TABLE IF NOT EXISTS building_metadata (
    id INTEGER NOT NULL,
    building_id TEXT NOT NULL,
    site_id TEXT NOT NULL,
    primary_space_usage TEXT NOT NULL,
    floor_area REAL NOT NULL,
    year_built INTEGER NOT NULL,
    has_solar TEXT NOT NULL DEFAULT 'No',
    PRIMARY KEY (id),
    UNIQUE (building_id, site_id)
);
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Main Campus', 'Academic', 240246, 1974);
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

transaction = """
SELECT
    id,
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,id,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,1,ECJ,UT Austin Main Campus,Academic,240246.0,1974,No


Notice that when inserting, we did not have to supply a value for the `id`? You may choose to supply one in fact but it is better practice to let the database engine autoincrement the `id` for you as more records are added:

In [29]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Pickle Research Campus', 'Laboratory', 740246, 1980)
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)
transaction = """
SELECT
    id,
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,id,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,1,ECJ,UT Austin Main Campus,Academic,240246.0,1974,No
1,2,ECJ,UT Austin Pickle Research Campus,Laboratory,740246.0,1980,No


We can also confirm that our `UNIQUE` constraint works as expected:

In [30]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Pickle Research Campus', 'Laboratory', 740246, 1980)
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

IntegrityError: UNIQUE constraint failed: building_metadata.building_id, building_metadata.site_id

What if we want to adhere to our `UNIQUE` constraint but be able to at least update values of other records in the case that our `UNIQUE` constraint is violated? The approach for this is to [UPSERT](https://www.sqlite.org/lang_upsert.html). When UPSERTING, you `INSERT` like you normally would but check for conflicts as a result of violated `UNIQUE` constraint, then handle the conflict in of two ways.

You can choose to either `DO NOTHING` where the `INSERT` statement has no effect i.e., nothing gets inserted nor updated. Think of this as a silent failure of the `UNIQUE` constraint violation. Aleternatively, you can choose to `UPDATE SET` which performs an `UPDATE` operation as fall back for being unable to `INSERT` a new record.

See the example below for the `DO NOTHING` choice:

In [31]:
read_table_transaction = """
SELECT
    id,
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
print('table before UPSERT:')
display(read_database(DATABASE_FILEPATH, read_table_transaction))

transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Pickle Research Campus', 'Laboratory', 740246, 2000)
ON CONFLICT (building_id, site_id) DO NOTHING
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

print('table after UPSERT:')
display(read_database(DATABASE_FILEPATH, read_table_transaction))

table before UPSERT:


,id,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,1,ECJ,UT Austin Main Campus,Academic,240246.0,1974,No
1,2,ECJ,UT Austin Pickle Research Campus,Laboratory,740246.0,1980,No


table after UPSERT:


,id,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,1,ECJ,UT Austin Main Campus,Academic,240246.0,1974,No
1,2,ECJ,UT Austin Pickle Research Campus,Laboratory,740246.0,1980,No


The record with `('ECJ', 'UT Austin Pickle Research Campus') for (building_id, site_id)` remained unchanged. See what happens with the alternative:

In [32]:
read_table_transaction = """
SELECT
    id,
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
print('table before UPSERT:')
display(read_database(DATABASE_FILEPATH, read_table_transaction))

transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Pickle Research Campus', 'Laboratory', 740246, 2000)
ON CONFLICT (building_id, site_id) DO UPDATE SET (primary_space_usage, floor_area, year_built) 
    = (EXCLUDED.primary_space_usage, EXCLUDED.floor_area, EXCLUDED.year_built)
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

print('table after UPSERT:')
display(read_database(DATABASE_FILEPATH, read_table_transaction))

table before UPSERT:


,id,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,1,ECJ,UT Austin Main Campus,Academic,240246.0,1974,No
1,2,ECJ,UT Austin Pickle Research Campus,Laboratory,740246.0,1980,No


table after UPSERT:


,id,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,1,ECJ,UT Austin Main Campus,Academic,240246.0,1974,No
1,2,ECJ,UT Austin Pickle Research Campus,Laboratory,740246.0,2000,No


We see that the `year_built` value is updated despite our `INSERT` statement violating the `(building_id, site_id)` `UNIQUE` constraint.

### The `CHECK` constraint

How about ensuring that the values entered are always one out of a set of values we expect. For example, the `has_solar` is binary and expected to have a value of `Yes` or `No` but as it stands, nothing stops us from inserting something totally unrelated or ambiguous such as `Maybe`. We can further improve our `TABLE` definition to catch these problems using the `CHECK` keyword. 

The original data stores the `has_solar` column as text where buildings that have solar have value of `Yes`. However, using your knowledge of how this feature will be used down the line can help inform you about how you should type-cast your data before writing to a database. A boolean variable will be better suited however, SQLite does not support boolean type and an integer is the closest similarity to boolean. To ensure that we keep the `has_solar` column binary, we use the `CHECK` keyword to enforce a rule that values entered for the column are either 0 or 1:

In [33]:
transaction = """
DROP TABLE IF EXISTS building_metadata;
CREATE TABLE IF NOT EXISTS building_metadata (
    id INTEGER NOT NULL,
    building_id TEXT NOT NULL,
    site_id TEXT NOT NULL,
    primary_space_usage TEXT NOT NULL,
    floor_area REAL NOT NULL,
    year_built INTEGER NOT NULL,
    has_solar INTEGER NOT NULL DEFAULT 0,
    PRIMARY KEY (id),
    UNIQUE (building_id, site_id),
    CONSTRAINT check_has_solar_boolean CHECK (has_solar IN (0, 1))
);
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Main Campus', 'Academic', 240246, 1974)
ON CONFLICT (building_id, site_id) DO NOTHING;
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

transaction = """
SELECT
    id,
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,id,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,1,ECJ,UT Austin Main Campus,Academic,240246.0,1974,0


If we try to insert a value that is neither 0 nor 1 as `has_solar`, we will get an error:

In [34]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built, has_solar)
VALUES
    ('ETC', 'UT Austin Main Campus', 'Academic', 240246, 1974, 5)
ON CONFLICT (building_id, site_id) DO NOTHING;
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

IntegrityError: CHECK constraint failed: check_has_solar_boolean

This feature is great because now we can ensure that we only insert valid values! We can include many other checks like ensuring that the `floor_area` and `year_built` are greater than 0 and that the site_id is one out of a list of values:

In [35]:
transaction = """
DROP TABLE IF EXISTS building_metadata;
CREATE TABLE IF NOT EXISTS building_metadata (
    id INTEGER NOT NULL,
    building_id TEXT NOT NULL,
    site_id TEXT NOT NULL,
    primary_space_usage TEXT NOT NULL,
    floor_area REAL NOT NULL,
    year_built INTEGER NOT NULL,
    has_solar INTEGER NOT NULL DEFAULT 0,
    PRIMARY KEY (id),
    UNIQUE (building_id, site_id),
    CONSTRAINT check_has_solar_boolean CHECK (has_solar IN (0, 1))
    CONSTRAINT check_floor_area_positive CHECK (floor_area > 0.0),
    CONSTRAINT check_year_built_positive CHECK (year_built > 0)
);
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built)
VALUES
    ('ECJ', 'UT Austin Main Campus', 'Academic', 240246, 1974)
ON CONFLICT (building_id, site_id) DO NOTHING;
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

transaction = """
SELECT
    id,
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,id,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,1,ECJ,UT Austin Main Campus,Academic,240246.0,1974,0


In [36]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built, has_solar)
VALUES
    ('ETC', 'UT Austin Main Campus', 'Academic', -40, 1974, 1)
ON CONFLICT (building_id, site_id) DO NOTHING;
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

IntegrityError: CHECK constraint failed: check_floor_area_positive

In [37]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built, has_solar)
VALUES
    ('ETC', 'UT Austin Main Campus', 'Academic', 240246, -2999, 1)
ON CONFLICT (building_id, site_id) DO NOTHING;
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

IntegrityError: CHECK constraint failed: check_year_built_positive

What if we want to enforce a check on the value of `site_id` so that the inserted value is always one value out of a finite set? We might choose to use the `CONSTRAINT CHECK` approach however, what happens if we choose to expand our database later on to include sites beyond what our database currently holds? Our `CONSTRAINT` will fail even though it is realistic for more sites to emerge.

In the same vein, we might insert a valid `site_id` but in a different casing from what other records have and it will go unchecked. See the example below:

In [38]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage, floor_area, year_built, has_solar)
VALUES
    ('ECJ', 'ut austin main campus', 'Academic', 240246, 1974, 1)
ON CONFLICT (building_id, site_id) DO NOTHING;
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)
transaction = """
SELECT
    id,
    building_id, 
    site_id, 
    primary_space_usage, 
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
;"""
display(read_database(DATABASE_FILEPATH, transaction))

,id,building_id,site_id,primary_space_usage,floor_area,year_built,has_solar
0,1,ECJ,UT Austin Main Campus,Academic,240246.0,1974,0
1,2,ECJ,ut austin main campus,Academic,240246.0,1974,1


The last insert very well violates our `UNIQUE` constraint on `(building_id, site_id)` but has gone unnoticed because we cleverly inserted the `site_id` in lowercase! We will also have issues if we tried to execute an `UPDATE` query to replace the `primary_space_usage` value for all records `WHERE site_id = 'UT Austin Main Campus'`. These kinds of data integrity issues can be avoided through [database normalization](https://en.wikipedia.org/wiki/Database_normalization). 

Without going into much detail, simply put, normalization requires that we create new tables out of the current table to ensure we don't have duplicated text but instead use `FOREIGN KEY` to reference text that will otherwise be duplicated in a `TABLE`. 

In the next section we will introduce the use of the `FOREIGN KEY` to help with data normalization.

### The `FOREIGN KEY` constraint to create relations

The `FOREIGN KEY` constraint states that a column in a table is in fact referencing the `PRIMARY KEY` of another `TABLE`. The syntax used to define a `FOREIGN KEY` is `FOREIGN KEY (<column_name>) REFERENCES <referenced_table_name> (<referenced_table_primary_key_column_name>) ON DELETE <action> ON UPDATE <action>`. The `ON DELETE` `<action>` determines what happens if a record whose primary key is used in the referencing table as a foreign key get is deleted in the referenced table. Whereas, the `ON UPDATE` `<action>` determines what happens to the `FOREIGN KEY` in the referencing table if the `PRIMARY KEY` in the referenced table changes. 

The `<action>` can be set to any of the following:
- `SET NULL`: When the primary key in the referenced table is deleted or updated, the foreign key in referencing table is set to `NULL`
- `SET DEFAULT`: When the primary key in the referenced table is deleted or updated, the foreign key in referencing table is set to a predefined default value.
- `RESTRICT`: This action prevents any delete or update to the primary key in referenced table unless all relations that reference it are removed.
- `NO ACTION`: Same as `RESTRICT`
- `CASCADE`: Propagates any change to the referenced table's primary key to all foreign key references.

From experience, it is best to set the `ON DELETE` `<action>` to `NO ACTION` and the `ON UPDATE` `<action>` to `CASCADE`. Although, in practice, `PRIMARY KEY` does not change after it is set so the `ON UPDATE` `<action>` is seldom triggered. 

To normalize our existing `building_metadata` `TABLE`, we want to define new tables for columns that will otherwise have repeating text. These are the `building_id`, `site_id` and `primary_space_usage` columns. These tables will only store the unique names of buildings, sites and primary space usages. Then, their `PRIMARY KEY` will be entered into our redesigned `building_metadata` `TABLE` as `FOREIGN KEY` constraints. Let us go ahead to define these new tables:

In [39]:
transaction = """
DROP TABLE IF EXISTS building_metadata;
DROP TABLE IF EXISTS building;
DROP TABLE IF EXISTS site;
DROP TABLE IF EXISTS primary_space_usage;
CREATE TABLE IF NOT EXISTS building (
    id INTEGER NOT NULL,
    name TEXT NOT NULL,
    PRIMARY KEY (id),
    UNIQUE (name)
);
CREATE TABLE IF NOT EXISTS site (
    id INTEGER NOT NULL,
    name TEXT NOT NULL,
    PRIMARY KEY (id),
    UNIQUE (name)
);
CREATE TABLE IF NOT EXISTS primary_space_usage (
    id INTEGER NOT NULL,
    name TEXT NOT NULL,
    PRIMARY KEY (id),
    UNIQUE (name)
);
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

Notice that we apply a `UNIQUE` constraint to each table's `name` column?

Also, notice that we `DROP` the `building_metadata` `TABLE` first before the other tables? Although we have not redefined `building_metadata` at this point with `FOREIGN KEY` constraints point to the other three tables, this is good practice in case we were to rerun this cell again after finally defining the new `building_metadata` table. If we do not `DROP` `building_metadata` first but attempt to `DROP` any of the other three, we will get an error because the other three tables will have a dependent table, `building_metadata`.

With the new tables defined, we can then insert unique building, site and primary space usage names:

In [40]:
transaction = """
INSERT INTO building (name)
VALUES 
    ('ECJ'),
    ('ETC')
ON CONFLICT (name) DO NOTHING
;
INSERT INTO site (name)
VALUES
    ('UT Austin Main Campus'),
    ('UT Austin Pickle Research Campus')
ON CONFLICT (name) DO NOTHING
;
INSERT INTO primary_space_usage (name)
VALUES
    ('Academic'),
    ('Laboratory')
ON CONFLICT (name) DO NOTHING
;
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

print('building table:')
display(read_database(DATABASE_FILEPATH, """SELECT name FROM building;"""))

print('site table:')
display(read_database(DATABASE_FILEPATH, """SELECT name FROM site;"""))

print('primary_space_usage table:')
display(read_database(DATABASE_FILEPATH, """SELECT name FROM primary_space_usage;"""))

building table:


,name
0,ECJ
1,ETC


site table:


,name
0,UT Austin Main Campus
1,UT Austin Pickle Research Campus


primary_space_usage table:


,name
0,Academic
1,Laboratory


We will now define the new schema for our `building_metadata` `TABLE` where we use `FOREIGN KEY` constraints:

In [41]:
transaction = """
DROP TABLE IF EXISTS building_metadata;
CREATE TABLE IF NOT EXISTS building_metadata (
    id INTEGER NOT NULL,
    building_id INTEGER NOT NULL,
    site_id INTEGER NOT NULL,
    primary_space_usage_id INTEGER NOT NULL,
    floor_area REAL NOT NULL,
    year_built INTEGER NOT NULL,
    has_solar INTEGER NOT NULL DEFAULT 0,
    PRIMARY KEY (id),
    FOREIGN KEY (building_id) REFERENCES building (id)
        ON DELETE NO ACTION
        ON UPDATE CASCADE,
    FOREIGN KEY (site_id) REFERENCES site (id)
        ON DELETE NO ACTION
        ON UPDATE CASCADE,
    FOREIGN KEY (primary_space_usage_id) REFERENCES primary_space_usage (id)
        ON DELETE NO ACTION
        ON UPDATE CASCADE,
    UNIQUE (building_id, site_id),
    CONSTRAINT check_has_solar_boolean CHECK (has_solar IN (0, 1)),
    CONSTRAINT check_floor_area_positive CHECK (floor_area > 0.0),
    CONSTRAINT check_year_built_positive CHECK (year_built > 0)
);
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

Next we will populate the new `building_metadata` table with records. We want to use the auto-generated `PRIMARY KEY` values in the `building`, `site`, and `primary_space_usage` tables to set the `FOREIGN KEY` constraints in `building_metadata`. For our small table size, we know the IDs by looking at the DataFrame however, for larger tables, we might not know the `PRIMARY KEY` ID right away and sometimes the autoincrement counter may skip numbers so it is not worthwhile to keep track.

To address, this, we will use a `SELECT` sub-statement within the `INSERT` statement for each column where we need to retrieve am ID where the `SELECT` statement queries the referenced table the relevant ID of a given unique name. See the example:

In [42]:
transaction = """
INSERT INTO building_metadata (building_id, site_id, primary_space_usage_id, floor_area, year_built, has_solar)
VALUES
    (
        (SELECT id FROM building WHERE name = 'ECJ'), 
        (SELECT id FROM site WHERE name = 'UT Austin Main Campus'), 
        (SELECT id FROM primary_space_usage WHERE name = 'Academic'), 
        240246, 
        1974, 
        1
    ),
    (
        (SELECT id FROM building WHERE name = 'ECJ'), 
        (SELECT id FROM site WHERE name = 'UT Austin Pickle Research Campus'), 
        (SELECT id FROM primary_space_usage WHERE name = 'Academic'), 
        543921, 
        1990, 
        0
    ),
    (
        (SELECT id FROM building WHERE name = 'ETC'), 
        (SELECT id FROM site WHERE name = 'UT Austin Pickle Research Campus'), 
        (SELECT id FROM primary_space_usage WHERE name = 'Laboratory'), 
        143567, 
        2000, 
        1
    )
ON CONFLICT (building_id, site_id) DO NOTHING
;
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

Finally, we can view the populated `building_metadata` `TABLE`:

In [43]:
transaction = """
SELECT
    id,
    building_id,
    site_id,
    primary_space_usage_id,
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
"""
display(read_database(DATABASE_FILEPATH, transaction))

,id,building_id,site_id,primary_space_usage_id,floor_area,year_built,has_solar
0,1,1,1,1,240246.0,1974,1
1,2,1,2,1,543921.0,1990,0
2,3,2,2,2,143567.0,2000,1


As-is, the `building_metadata` `TABLE` has no clear meaning because we do not know what the `INTEGER` values for `building_id`, `site_id`, and `primary_space_usage_id` mean but we know that the information we need is stored in our `building`, `site` and `primary_space_usage` tables.

In the next section, we will introduce the `JOIN` keyword that allows us merge data from different tables to return a single representation.

## Joins
---

Broadly speaking there are five types of joins in SQL as illustrated and described in the figure:

<figure class="image">
  <img src="https://storage.googleapis.com/gweb-cloudblog-publish/images/joins_1.max-1600x1600.png"  width="700" alt="SQL join types.">
  <figcaption>SQL join types (<a href="https://cloud.google.com/blog/topics/developers-practitioners/bigquery-explained-working-joins-nested-repeated-data">Thallam, 2020</a>).</figcaption>
</figure>

SQLite supports only `INNER JOIN`, `LEFT JOIN`, and `CROSS JOIN`. We will use joins to retrieve a view of our `building_metadata` table that is reminiscent of what it looked like before normalization. In the example below, we `LEFT JOIN` `building`, `site`, and `primary_space_usage` to `building_metadata`:

In [44]:
transaction = """
SELECT
    b.name AS building,
    s.name AS site,
    p.name AS primary_space_usage,
    m.floor_area, 
    m.year_built, 
    m.has_solar
FROM building_metadata m
LEFT JOIN building b ON b.id = m.building_id
LEFT JOIN site s ON s.id = m.site_id
LEFT JOIN primary_space_usage p ON p.id = m.primary_space_usage_id
"""
display(read_database(DATABASE_FILEPATH, transaction))

,building,site,primary_space_usage,floor_area,year_built,has_solar
0,ECJ,UT Austin Main Campus,Academic,240246.0,1974,1
1,ECJ,UT Austin Pickle Research Campus,Academic,543921.0,1990,0
2,ETC,UT Austin Pickle Research Campus,Laboratory,143567.0,2000,1


There are few things to note in the above query. First, we have used the `<column_name> AS <name>` syntax to reassign names to the columns of the returned view. This is called aliasing. This is important because sometimes as in our case, joined tables might have columns with the same names.

Second, we have given all four tables one-character aliases. This serves two purposes:
1. It is convenient to reference the alias when qualifying a column instead of the full table name.
2. It is used to resolve name conflicts for tables that have a common column name of `name` that is referenced in the query.

Third, we have joined using the `PRIMARY KEY`-`FOREIGN KEY` relationships using the `ON` keyword. However, you don't necessarily have to take this approach. You can apply joins on any two columns in two tables that have similar values. The only criteria for a `JOIN` to work is that two records are joined using columns that have the same value. You can also `JOIN` on multiple conditions. Our current schema does not afford us the ability to show such example but think of a scenario where you have one table that holds a time series of energy consumption for different commodities. The table has 3 columns: timestamp, commodity, consumption. Then there is another table that is a time series of dynamic pricing for each commodity with three columns as well: timestamp, commodity, price. If you wanted to write a query that enables you calculate the cost of any commodity at any timestamp, such query will resemble:

```sql
SELECT
    c.timestamp,
    c.commodity,
    c.consumption*p.price AS cost
FROM table_1 c
INNER JOIN table_2 p ON 
    p.timestamp = c.timestamp 
    AND p.commodity = c.commodity
```

See how we have joined on both the timestamp and commodity columns? We also used an `INNER JOIN` so only commodities in table_1 that have price defined in table_2 will be returned. Also the returned view will include only records with timestamps that have price defined in table_2.

So far, we have returned all records in tables when we read from our database. However, more often than not, you only want to return a subset of the table. The `WHERE` keyword comes in handy as discussed in the next section.

## Conditional statements using `WHERE`
---

We have already seen the `WHERE` keyword applied when inserting the `PRIMARY KEY` of one table as the `FOREIGN KEY` in another. It can also be used in the `SELECT` statement to filter records that are not required. For example, we might want to return only records that are on the 'UT Austin Pickle Research Campus' site but built after 1990:

In [45]:
transaction = """
SELECT
    b.name AS building,
    s.name AS site,
    p.name AS primary_space_usage,
    m.floor_area, 
    m.year_built, 
    m.has_solar
FROM building_metadata m
LEFT JOIN building b ON b.id = m.building_id
LEFT JOIN site s ON s.id = m.site_id
LEFT JOIN primary_space_usage p ON p.id = m.primary_space_usage_id
WHERE 
    m.site_id = (SELECT id FROM site WHERE name = 'UT Austin Pickle Research Campus')
    AND m.year_built > 1990
"""
display(read_database(DATABASE_FILEPATH, transaction))

,building,site,primary_space_usage,floor_area,year_built,has_solar
0,ETC,UT Austin Pickle Research Campus,Laboratory,143567.0,2000,1


Alternatively, we may choose to retrieve only records where the site is either 'UT Austin Main Campus' or 'UT Austin Pickle Research Campus' but built before 2000:

In [46]:
transaction = """
SELECT
    b.name AS building,
    s.name AS site,
    p.name AS primary_space_usage,
    m.floor_area, 
    m.year_built, 
    m.has_solar
FROM building_metadata m
LEFT JOIN building b ON b.id = m.building_id
LEFT JOIN site s ON s.id = m.site_id
LEFT JOIN primary_space_usage p ON p.id = m.primary_space_usage_id
WHERE 
    m.site_id IN (SELECT id FROM site WHERE name IN ('UT Austin Main Campus', 'UT Austin Pickle Research Campus'))
    AND m.year_built < 2000
"""
display(read_database(DATABASE_FILEPATH, transaction))

,building,site,primary_space_usage,floor_area,year_built,has_solar
0,ECJ,UT Austin Main Campus,Academic,240246.0,1974,1
1,ECJ,UT Austin Pickle Research Campus,Academic,543921.0,1990,0


The commonly used keywords and comparison operators in the `WHERE` clause include: `=`, `!=`, `>`, `<`, `>=`, `<=`, `IN`, `NOT IN`, `IS NULL`,  `NOT NULL`, `BETWEEN <value_1> AND <value_2>`. Example applications are:

- `column_name = value`
- `column_name != value`
- `column_name > value`
- `column_name < value`
- `column_name >= value`
- `column_name <= value`
- `column_name IN (value_1, value_2, value_3)`
- `column_name NOT IN (value_1, value_2, value_3)`
- `column_name IS NULL`
- `column_name IS NOT NULL`
- `column_name BETWEEN value_1 AND value_2`

You can also compare expressions:

- `column_name*2 = value`
- `column_name_1 != column_name_2/4`
- `column_name_1 > value + column_name_2`

## Inserting many records
---

At this point in the tutorial, you have learned how to define a table as well as constraints that enforce the integrity of inserted values. You have also learned about normalization, joins and conditionally returning views of tables. We can now go back to our original dataset we loaded in the earlier section so that we insert all its records into the `building_metadata` table we have built:

In [47]:
display(building_metadata.head())

,building_id,site_id,primaryspaceusage,sqft,solar,yearbuilt
0,Panther_lodging_Dean,Panther,Lodging/residential,5477.0,NaN,1989.0
1,Panther_lodging_Shelia,Panther,Lodging/residential,10000.0,NaN,1992.0
2,Panther_lodging_Ricky,Panther,Lodging/residential,5200.0,NaN,2016.0
3,Panther_education_Rosalie,Panther,Education,7432.0,NaN,2008.0
4,Panther_education_Misty,Panther,Education,2720.0,NaN,2004.0


### Data integrity pre-check

Before inserting, we will need to carry out some data transformation including extracting the building from the `building_id` column, and converting the `solar` column to an `INTEGER` type:

In [48]:
# extract building name
building_metadata['building_name'] = building_metadata['building_id'].str.split('_', expand=True)[2]

# set solar to binary integer
building_metadata.loc[building_metadata['solar']=='Yes', 'solar'] = 1

# set solar value to 0 for records with NULL solar value
building_metadata['solar'] = building_metadata['solar'].fillna(0)

display(building_metadata.head())

,building_id,site_id,primaryspaceusage,sqft,solar,yearbuilt,building_name
0,Panther_lodging_Dean,Panther,Lodging/residential,5477.0,0,1989.0,Dean
1,Panther_lodging_Shelia,Panther,Lodging/residential,10000.0,0,1992.0,Shelia
2,Panther_lodging_Ricky,Panther,Lodging/residential,5200.0,0,2016.0,Ricky
3,Panther_education_Rosalie,Panther,Education,7432.0,0,2008.0,Rosalie
4,Panther_education_Misty,Panther,Education,2720.0,0,2004.0,Misty


We will also set the `building_name`, `site_id` and `primaryspaceusage` columns to capitalize only the first character to avoid situations where the same name is entered but with different casing:

In [49]:
for c in ['building_name', 'site_id', 'primaryspaceusage']:
    building_metadata[c] = building_metadata[c].str.capitalize()

display(building_metadata.head())

,building_id,site_id,primaryspaceusage,sqft,solar,yearbuilt,building_name
0,Panther_lodging_Dean,Panther,Lodging/residential,5477.0,0,1989.0,Dean
1,Panther_lodging_Shelia,Panther,Lodging/residential,10000.0,0,1992.0,Shelia
2,Panther_lodging_Ricky,Panther,Lodging/residential,5200.0,0,2016.0,Ricky
3,Panther_education_Rosalie,Panther,Education,7432.0,0,2008.0,Rosalie
4,Panther_education_Misty,Panther,Education,2720.0,0,2004.0,Misty


Next, we confirm that there are no duplicates of `building_name` - `site_id` pairs:

In [50]:
print('Duplicated building_name-site_id pairs:', building_metadata[building_metadata.duplicated(['building_name', 'site_id'])].shape[0])

Duplicated building_name-site_id pairs: 0


Lastly, we want to ensure that there are no `NULL` values and if there are figure out a way to handle them:

In [51]:
for c in ['building_name', 'site_id', 'primaryspaceusage', 'sqft', 'solar', 'yearbuilt']:
    print(f'{c} NULL count =', building_metadata[building_metadata[[c]].isnull().any(axis=1)].shape[0], 'of', building_metadata.shape[0])

building_name NULL count = 0 of 1636
site_id NULL count = 0 of 1636
primaryspaceusage NULL count = 21 of 1636
sqft NULL count = 0 of 1636
solar NULL count = 0 of 1636
yearbuilt NULL count = 819 of 1636


There are a few records with `NULL` `primaryspaceusage` but almost half the number of buildings do not have `yearbuilt` defined. We can assign a default value for missing `primaryspaceusage` such as 'Unknown' but we can't assume or guess the `yearbuilt`. Thus, we will remove the `NOT NULL` constraint from `year_built` in our `building_metadata` table to accommodate `NULL` values.

Let us begin by assigning a default `primaryspaceuse`:

In [52]:
DEFAULT_PRIMARY_SPACE_USE = 'Unknown'
building_metadata.loc[building_metadata['primaryspaceusage'].isnull(), 'primaryspaceusage'] = DEFAULT_PRIMARY_SPACE_USE
display(building_metadata[building_metadata['primaryspaceusage']==DEFAULT_PRIMARY_SPACE_USE])

,building_id,site_id,primaryspaceusage,sqft,solar,yearbuilt,building_name
1179,Swan_unknown_Reyna,Swan,Unknown,66247.0,0,1972.0,Reyna
1180,Swan_unknown_Douglas,Swan,Unknown,83272.0,0,1989.0,Douglas
1181,Swan_unknown_Darrin,Swan,Unknown,68026.0,0,1901.0,Darrin
1182,Swan_unknown_Ike,Swan,Unknown,74574.0,0,1903.0,Ike
1183,Swan_unknown_Esteban,Swan,Unknown,75093.0,0,1990.0,Esteban
1184,Swan_unknown_Christoper,Swan,Unknown,92964.0,0,1940.0,Christoper
1185,Swan_unknown_Isaiah,Swan,Unknown,159374.0,0,2000.0,Isaiah
1186,Swan_unknown_Tom,Swan,Unknown,49998.0,0,1986.0,Tom
1187,Swan_unknown_Jan,Swan,Unknown,160942.0,0,1996.0,Jan
1188,Swan_unknown_Andres,Swan,Unknown,124202.0,0,1998.0,Andres


Then we remove the `NOT NULL` constraint from `year_built`:

In [53]:
transaction = """
DROP TABLE IF EXISTS building_metadata;
CREATE TABLE IF NOT EXISTS building_metadata (
    id INTEGER NOT NULL,
    building_id INTEGER NOT NULL,
    site_id INTEGER NOT NULL,
    primary_space_usage_id INTEGER NOT NULL,
    floor_area REAL NOT NULL,
    year_built INTEGER,
    has_solar INTEGER NOT NULL DEFAULT 0,
    PRIMARY KEY (id),
    FOREIGN KEY (building_id) REFERENCES building (id)
        ON DELETE NO ACTION
        ON UPDATE CASCADE,
    FOREIGN KEY (site_id) REFERENCES site (id)
        ON DELETE NO ACTION
        ON UPDATE CASCADE,
    FOREIGN KEY (primary_space_usage_id) REFERENCES primary_space_usage (id)
        ON DELETE NO ACTION
        ON UPDATE CASCADE,
    UNIQUE (building_id, site_id),
    CONSTRAINT check_has_solar_boolean CHECK (has_solar IN (0, 1)),
    CONSTRAINT check_floor_area_positive CHECK (floor_area > 0.0),
    CONSTRAINT check_year_built_positive CHECK (year_built > 0)
);
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

### Insert records

There are 1,636 records in the `building_metadata` dataframe so it is rather tasking to manually write the `INSERT` statement that includes all 1,636 records. Instead we will use the `execute_many` function associated with the database to achieve this. We will write a convenience function that handles all the other stuff like closing the connection after executing the many insert statements:

In [54]:
def insert_many(database_filepath: str, statements: List[str], data_list: List[Tuple[Union[Tuple, Mapping]]]) -> Tuple[List[List[str]], List[List[tuple]]]:
    column_names = []
    records = []

    try:
        # connect to database
        database = sqlite3.connect(database_filepath)

        # cursor used to execute statements
        cursor = database.cursor()

        for s, d in zip(statements, data_list):
            cursor.executemany(s, d)

            # collect any data returned from the executed statement
            if cursor.description is not None:
                column_names.append([d[0] for d in cursor.description])
                records.append(cursor.fetchall())
            else:
                pass
        
        # commit the transaction to the database otherwise it does not
        # get saved when the database closes
        database.commit()
    
    finally:
        # close all connections
        cursor.close()
        database.close()

    return column_names, records

We will then go ahead to delete any existing records in the tables from earlier:

In [55]:
transaction = """
DELETE FROM building_metadata;
DELETE FROM building;
DELETE FROM site;
DELETE FROM primary_space_usage;
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

Notice that we delete from the `building_metadata` `TABLE` first before the other tables? This is because we set our `ON DELETE` action in `building_metadata` for all `FOREIGN KEY` constraints to `NO ACTION` so not until the references to `PRIMARY KEY` constraints in `building`, `site`, and `primary_space_usage` seize to exist can we delete records in these three tables.

Finally, we insert values in all tables:

In [56]:
transaction = """
INSERT INTO building (name) VALUES (:building_name);
INSERT INTO site (name) VALUES (:site_id);
INSERT INTO primary_space_usage (name) VALUES (:primaryspaceusage);
INSERT INTO building_metadata (building_id, site_id, primary_space_usage_id, floor_area, year_built, has_solar)
    VALUES (
        (SELECT id FROM building WHERE name = :building_name),
        (SELECT id FROM site WHERE name = :site_id),
        (SELECT id FROM primary_space_usage WHERE name = :primaryspaceusage),
        :sqft,
        :yearbuilt,
        :solar
    );
"""
statements = transaction.split(';')
data_list = [
    building_metadata[['building_name']].drop_duplicates().to_dict('records'),
    building_metadata[['site_id']].drop_duplicates().to_dict('records'),
    building_metadata[['primaryspaceusage']].drop_duplicates().to_dict('records'),
    building_metadata.to_dict('records')
]
_ = insert_many(DATABASE_FILEPATH, statements, data_list)

Above, we have used placeholder to [bind](https://docs.python.org/3/library/sqlite3.html#how-to-use-placeholders-to-bind-values-in-sql-queries) column names in the dataframe to column names in the respective database tables. Using the `insert_many` function we built, we can execute many insert statements in one transaction. Remember what we learned about transaction in the beginning that it is an all or nothing affair. If any one statement in the transaction fails, the entire transaction fails. This is the behavior we want because we want to ensure that we are able to write to all tables. 

Additionally, we do not perform any `UPSERT` because we had just deleted all records before the `INSERT` statements and also dropped duplicates before supplying data to be inserted in the `building`, `site` and `primary_space_usage` tables. Hence, we do not expect any conflicts and if there are, we do not want them to pass silently.

In our transaction, we have four `INSERT` statements thus, we populate four dictionaries in the `data_list` that provide `INSERT` values for each `INSERT` statement.

We can now go ahead and inspect the state of the tables after the `INSERT` transaction:

In [57]:
print('building table:')
display(read_database(DATABASE_FILEPATH, """SELECT name FROM building;""").head())

print('site table:')
display(read_database(DATABASE_FILEPATH, """SELECT name FROM site;""").head())

print('primary_space_usage table:')
display(read_database(DATABASE_FILEPATH, """SELECT name FROM primary_space_usage;""").head())

print('building_metadata table:')
display(read_database(DATABASE_FILEPATH, """
SELECT
    id,
    building_id,
    site_id,
    primary_space_usage_id,
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
""").head())

building table:


,name
0,Dean
1,Shelia
2,Ricky
3,Rosalie
4,Misty


site table:


,name
0,Panther
1,Robin
2,Fox
3,Rat
4,Bear


primary_space_usage table:


,name
0,Lodging/residential
1,Education
2,Office
3,Entertainment/public assembly
4,Other


building_metadata table:


,id,building_id,site_id,primary_space_usage_id,floor_area,year_built,has_solar
0,1,1,1,1,5477.0,1989.0,0
1,2,2,1,1,10000.0,1992.0,0
2,3,3,1,1,5200.0,2016.0,0
3,4,4,1,2,7432.0,2008.0,0
4,5,5,1,2,2720.0,2004.0,0


## Aggregation
---

Now we have our tables populated, we will create aggregations that provide summary statistics of our data. SQLite has the basic aggregations that are typically carried out including:

- `AVG(X)`: Average of all non `NULL` `X` values
- `COUNT(X)`: Number of times `X` is non `NULL` 
- `STRING_AGG(X, Y)`: Concatenation of all non `NULL` `X` values separated by `Y` 
- `MAX(X)`: Maximum `X` value
- `MIN(X)`: Minimum `X` value
- `SUM(X)`: Sum of all non `NULL` `X` values. Returns `NULL` if all values are `NULL`
- `TOTAL(X)`: Same as `SUM` but returns `0` if all values are `NULL`

When applying aggregations, we use the `GROUP BY` keywords to specify how we want to partition the data before aggregating. Any column not included in the `GROUP BY` but shows up in the selected columns must have an aggregation function acting on it.

See the example below where we get the summary statistics of the `floor_area` for each site in `building_metadata`:

In [58]:
transaction = """
SELECT
    s.name AS site,
    AVG(m.floor_area) AS average_floor_area,
    MIN(m.floor_area) AS minimum_floor_area,
    MAX(m.floor_area) AS maximum_floor_area,
    SUM(m.floor_area) AS total_floor_area
FROM building_metadata m
LEFT JOIN site s ON s.id = m.site_id
GROUP BY
    s.name
"""
display(read_database(DATABASE_FILEPATH, transaction).head())

,site,average_floor_area,minimum_floor_area,maximum_floor_area,total_floor_area
0,Bear,94485.021739,387.0,420885.0,8692622.0
1,Bobcat,73977.833333,4482.0,213762.0,2663202.0
2,Bull,144874.209677,5682.0,745671.0,17964402.0
3,Cockatoo,83921.387097,4071.0,322341.0,10406252.0
4,Crow,104563.400000,68030.0,152559.0,522817.0


The next example returns the `year_built` summary statistics for each `primary_space_usage` per site but only for instances where the `year_built` is `NOT NULL`. we will also order the results in ascending order of site name followed by descending order of average `year_built` and limit our returned view to only the first 5 records:

In [59]:
transaction = """
SELECT
    s.name AS site,
    p.name AS primary_space_usage,
    AVG(m.year_built) AS average_year_built,
    MIN(m.year_built) AS minimum_year_built,
    MAX(m.year_built) AS maximum_year_built
FROM building_metadata m
LEFT JOIN site s ON s.id = m.site_id
LEFT JOIN primary_space_usage p ON p.id = m.primary_space_usage_id
WHERE m.year_built IS NOT NULL
GROUP BY
    s.name,
    p.name
ORDER BY
    s.name ASC,
    AVG(m.year_built) DESC
LIMIT 5
"""
display(read_database(DATABASE_FILEPATH, transaction).head())

,site,primary_space_usage,average_year_built,minimum_year_built,maximum_year_built
0,Bear,Utility,1999.000000,1999,1999
1,Bear,Parking,1983.666667,1955,2007
2,Bear,Public services,1960.666667,1917,2004
3,Bear,Technology/science,1952.500000,1942,1963
4,Bear,Education,1952.323077,1905,2015


We have introduced a couple of new keywords in the query above. `ORDER BY` allows us sort the returned view using any combination of columns. Notice that to sort by average `year_built`, we use the aggregation function call on the column and not the column name?

The `LIMIT` keyword returns the first `n` records.

You can learn about [other components of the SELECT statement](https://www.sqlite.org/lang_select.html).

### The `DISTINCT` keyword

The `DISTINCT` keyword is not an aggregation function but is very useful for quickly retrieving unique column values or unique sets of column values. The example provided below returns the unique set of site and primary space usage names:

In [60]:
transaction = """
SELECT DISTINCT
    s.name AS site,
    p.name AS primary_space_usage
FROM building_metadata m
LEFT JOIN site s ON s.id = m.site_id
LEFT JOIN primary_space_usage p ON p.id = m.primary_space_usage_id
"""
display(read_database(DATABASE_FILEPATH, transaction))

,site,primary_space_usage
0,Panther,Lodging/residential
1,Panther,Education
2,Panther,Office
3,Panther,Entertainment/public assembly
4,Panther,Other
...,...,...
114,Cockatoo,Utility
115,Cockatoo,Healthcare
116,Mouse,Healthcare
117,Mouse,Lodging/residential


We could have chosen to use `GROUP BY` to arrive at the same result while also counting the number of times the pairs occur:

In [61]:
transaction = """
SELECT DISTINCT
    s.name AS site,
    p.name AS primary_space_usage,
    COUNT(p.name) AS "count"
FROM building_metadata m
LEFT JOIN site s ON s.id = m.site_id
LEFT JOIN primary_space_usage p ON p.id = m.primary_space_usage_id
GROUP BY
    s.name,
    p.name
ORDER BY
    COUNT(p.name) DESC
"""
display(read_database(DATABASE_FILEPATH, transaction))

,site,primary_space_usage,count
0,Rat,Education,111
1,Rat,Public services,93
2,Lamb,Education,83
3,Hog,Office,77
4,Bear,Education,67
...,...,...,...
114,Rat,Religious worship,1
115,Rat,Retail,1
116,Robin,Entertainment/public assembly,1
117,Wolf,Public services,1


We end up with the same 119 records as before. There is always more than one way to arrive at the same view when writing queries. The approach you choose is usually guided by writing style, level of brevity and cost of query in terms of time it takes to execute the query and return results. Initially, you typically start with something that just works! As your data grows in size, there might be a need to optimize your query to reduce its cost. Good design is to anticipate how your data will scale and design your initial queries to accommodate accordingly.

## Creating a View
---

We have carried out `SELECT` statements while specifying conditions that filter out certain records. We have also seen how we can order the returned results or aggregate columns to extract summary statistics. There are some `SELECT` statements that we may use frequently that we do not want to have to write every time we need. The `VIEW` statement allows us to assign a name to `SELECT` statement that is then permanently stored in the database schema until deleted. The typical syntax for defining a view is `CREATE VIEW IF NOT EXISTS <view_name> AS <select_statement>`.

Let us now create a view for our last aggregation `SELECT` statement:

In [62]:
transaction = """
CREATE VIEW IF NOT EXISTS primary_space_usage_count_per_site AS
    SELECT DISTINCT
        s.name AS site,
        p.name AS primary_space_usage,
        COUNT(p.name) AS "count"
    FROM building_metadata m
    LEFT JOIN site s ON s.id = m.site_id
    LEFT JOIN primary_space_usage p ON p.id = m.primary_space_usage_id
    GROUP BY
        s.name,
        p.name
    ORDER BY
        COUNT(p.name) DESC
;"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

We can now select from the `VIEW` as if it were a `TABLE`:

In [63]:
transaction = """
SELECT
    site,
    primary_space_usage,
    count
FROM primary_space_usage_count_per_site
"""
display(read_database(DATABASE_FILEPATH, transaction))

,site,primary_space_usage,count
0,Rat,Education,111
1,Rat,Public services,93
2,Lamb,Education,83
3,Hog,Office,77
4,Bear,Education,67
...,...,...,...
114,Rat,Religious worship,1
115,Rat,Retail,1
116,Robin,Entertainment/public assembly,1
117,Wolf,Public services,1


In SQLite, a `VIEW` cannot be modified after definition and must be dropped first if it is to be redefined. Also, as with dropping tables that have dependents through`FOREIGN KEY` links, views cannot be dropped until all dependent views have been dropped. This is an important thing to note as you build complex schemas as you may not be able to change a view's definition because of one-too-many dependents that have to be dropped that may have dependents of their own! The better practice is to keep the use of views within views at a minimum.

In the next cell, we will see how to `DROP` a view:

In [64]:
_ = execute_transaction(DATABASE_FILEPATH, "DROP VIEW IF EXISTS primary_space_usage_count_per_site;")

Now, if we try to query the view, we will get an error message:

In [65]:
transaction = """
SELECT
    site,
    primary_space_usage,
    count
FROM primary_space_usage_count_per_site
"""
display(read_database(DATABASE_FILEPATH, transaction))

OperationalError: no such table: primary_space_usage_count_per_site

## Bringing it all in
---

In this final section, we will define one schema that contains all our `TABLE` and `VIEW` definitions and re-insert values into our tables.

First let us define the schema:

In [66]:
transaction = """
DROP VIEW IF EXISTS primary_space_usage_count_per_site;
DROP TABLE IF EXISTS building_metadata;
DROP TABLE IF EXISTS building;
DROP TABLE IF EXISTS site;
DROP TABLE IF EXISTS primary_space_usage;
CREATE TABLE building (
    id INTEGER NOT NULL,
    name TEXT NOT NULL,
    PRIMARY KEY (id),
    UNIQUE (name)
);
CREATE TABLE site (
    id INTEGER NOT NULL,
    name TEXT NOT NULL,
    PRIMARY KEY (id),
    UNIQUE (name)
);
CREATE TABLE primary_space_usage (
    id INTEGER NOT NULL,
    name TEXT NOT NULL,
    PRIMARY KEY (id),
    UNIQUE (name)
);
CREATE TABLE building_metadata (
    id INTEGER NOT NULL,
    building_id INTEGER NOT NULL,
    site_id INTEGER NOT NULL,
    primary_space_usage_id INTEGER NOT NULL,
    floor_area REAL NOT NULL,
    year_built INTEGER,
    has_solar INTEGER NOT NULL DEFAULT 0,
    PRIMARY KEY (id),
    FOREIGN KEY (building_id) REFERENCES building (id)
        ON DELETE NO ACTION
        ON UPDATE CASCADE,
    FOREIGN KEY (site_id) REFERENCES site (id)
        ON DELETE NO ACTION
        ON UPDATE CASCADE,
    FOREIGN KEY (primary_space_usage_id) REFERENCES primary_space_usage (id)
        ON DELETE NO ACTION
        ON UPDATE CASCADE,
    UNIQUE (building_id, site_id),
    CONSTRAINT check_has_solar_boolean CHECK (has_solar IN (0, 1)),
    CONSTRAINT check_floor_area_positive CHECK (floor_area > 0.0),
    CONSTRAINT check_year_built_positive CHECK (year_built > 0)
);
CREATE VIEW primary_space_usage_count_per_site AS
    SELECT DISTINCT
        s.name AS site,
        p.name AS primary_space_usage,
        COUNT(p.name) AS "count"
    FROM building_metadata m
    LEFT JOIN site s ON s.id = m.site_id
    LEFT JOIN primary_space_usage p ON p.id = m.primary_space_usage_id
    GROUP BY
        s.name,
        p.name
    ORDER BY
        COUNT(p.name) DESC
;
"""
_ = execute_transaction(DATABASE_FILEPATH, transaction)

We then `INSERT` records into the tables like before:

In [67]:
transaction = """
INSERT INTO building (name) VALUES (:building_name);
INSERT INTO site (name) VALUES (:site_id);
INSERT INTO primary_space_usage (name) VALUES (:primaryspaceusage);
INSERT INTO building_metadata (building_id, site_id, primary_space_usage_id, floor_area, year_built, has_solar)
    VALUES (
        (SELECT id FROM building WHERE name = :building_name),
        (SELECT id FROM site WHERE name = :site_id),
        (SELECT id FROM primary_space_usage WHERE name = :primaryspaceusage),
        :sqft,
        :yearbuilt,
        :solar
    );
"""
statements = transaction.split(';')
data_list = [
    building_metadata[['building_name']].drop_duplicates().to_dict('records'),
    building_metadata[['site_id']].drop_duplicates().to_dict('records'),
    building_metadata[['primaryspaceusage']].drop_duplicates().to_dict('records'),
    building_metadata.to_dict('records')
]
_ = insert_many(DATABASE_FILEPATH, statements, data_list)

... and for one last time, visualize the tables and view:

In [68]:
print('building table:')
display(read_database(DATABASE_FILEPATH, """SELECT name FROM building;""").head())

print('site table:')
display(read_database(DATABASE_FILEPATH, """SELECT name FROM site;""").head())

print('primary_space_usage table:')
display(read_database(DATABASE_FILEPATH, """SELECT name FROM primary_space_usage;""").head())

print('building_metadata table:')
display(read_database(DATABASE_FILEPATH, """
SELECT
    id,
    building_id,
    site_id,
    primary_space_usage_id,
    floor_area, 
    year_built, 
    has_solar
FROM building_metadata
""").head())

print('primary_space_usage_count_per_site view:')
display(read_database(DATABASE_FILEPATH, """
SELECT
    site,
    primary_space_usage,
    count
FROM primary_space_usage_count_per_site
""").head())

building table:


,name
0,Dean
1,Shelia
2,Ricky
3,Rosalie
4,Misty


site table:


,name
0,Panther
1,Robin
2,Fox
3,Rat
4,Bear


primary_space_usage table:


,name
0,Lodging/residential
1,Education
2,Office
3,Entertainment/public assembly
4,Other


building_metadata table:


,id,building_id,site_id,primary_space_usage_id,floor_area,year_built,has_solar
0,1,1,1,1,5477.0,1989.0,0
1,2,2,1,1,10000.0,1992.0,0
2,3,3,1,1,5200.0,2016.0,0
3,4,4,1,2,7432.0,2008.0,0
4,5,5,1,2,2720.0,2004.0,0


primary_space_usage_count_per_site view:


,site,primary_space_usage,count
0,Rat,Education,111
1,Rat,Public services,93
2,Lamb,Education,83
3,Hog,Office,77
4,Bear,Education,67


## Conclusion
---

We have learned some of the basics of relational databases using the SQLite engine. However, there is so much more to relational databases and SQLite than what is covered in this notebook.

Thus it is encouraged that you take some time to read through the references and further reading links. Most importantly, practice, practice, practice!

## Further reading and references
---

1. [What Is a Database?](https://www.oracle.com/database/what-is-database/#:~:text=Is%20a%20Database%3F-,Database%20defined,database%20management%20system%20(DBMS).)
2. [SQLite Tutorial](https://www.sqlitetutorial.net)
4. [Introduction to Python SQL Libraries](https://realpython.com/python-sql-libraries/)
5. [Data Management With Python, SQLite, and SQLAlchemy](https://realpython.com/python-sqlite-sqlalchemy/)
3. [sqlite3 — DB-API 2.0 interface for SQLite databases](https://docs.python.org/3/library/sqlite3.html#module-sqlite3)
6. [Official SQLite Website](https://www.sqlite.org/index.html)